# Test

In [1]:
!pip install pydub

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [9]:
from pydub import AudioSegment

def change_pitch(input_file, output_file):
    # WAV 파일 불러오기
    sound = AudioSegment.from_wav(input_file)

    # 피치 2배로 변경
    new_sound = sound.speedup(playback_speed=1.1)

    # 변경된 파일 저장
    new_sound.export(output_file, format="wav")

def get_wav_duration(file_path):
    # WAV 파일 불러오기
    sound = AudioSegment.from_wav(file_path)

    # 길이 계산 (밀리초 단위)
    duration_in_milliseconds = len(sound)

    # 밀리초를 초로 변환
    duration_in_seconds = duration_in_milliseconds / 1000

    return duration_in_seconds


if __name__ == "__main__":
    input_file = "/content/VocalRemover5-COLAB_arch/separated/4_Vocals.wav"  # 입력 WAV 파일 경로
    output_file = "/content/VocalRemover5-COLAB_arch/separated/4_Vocals_2pitch.wav"  # 출력 WAV 파일 경로

    #change_pitch(input_file, output_file)
    sec = get_wav_duration(input_file)
    minit = sec//60
    sec = round(sec%60, 2)
    print(minit, '분', sec, '초')

2.0 분 31.74 초


# Generate Dataset (Isolate Vocals)

## Setting Environment

In [1]:
#@markdown ### Input files
#@markdown Track name / any link (Upload your songs in tracks folder)
#input = "https://www.youtube.com/watch?v=nSzwuS-mvkk&t=90s" #@param {type:"string"}
input = "https://www.youtube.com/watch?v=EVJjmMW7eII"

In [2]:
import os
os.makedirs('/content', exist_ok=True)
os.chdir('/content')
# ------------VSC REWRITE
# pyright: reportMissingImports=false, reportUnusedVariable=warning, reportUntypedBaseClass=error
from IPython.display import Audio, display
#from google.colab import output
#from google.colab import drive
#from google.colab import files
from sys import exit
import zipfile
import hashlib
import os.path
import shutil
import psutil
import random
import glob
import time
import zlib
import sys
import torch 
from pathvalidate import sanitize_filename # for inference cell
import yt_dlp as youtube_dl # for inference cell

isCPU = torch.cuda.is_available()
#@markdown Uncheck if you want to use VocalRemover5 without mounting to drive.
MountDrive = False #@param{type:"boolean"}
#@markdown Use mounting method
method = 'new' #@param ["new","old"]
#@markdown Mounting path; don't touch this if you don't know what you're doing
mounting_path = '/content/drive/MyDrive' #@param ["snippets:","/content/drive/MyDrive","/content/drive/Shareddrives/<your shared drive name>", "/content/drive/Shareddrives/Shared Drive"]{allow-input: true}
#@markdown Force trigger update
ForceUpdate = False #@param{type:"boolean"}

#update channels
ai = 'https://github.com/NaJeongMo/Colaboratory-Notebook-for-Ultimate-Vocal-Remover'
vercheck = 'https://raw.githubusercontent.com/NaJeongMo/Colaboratory-Notebook-for-Ultimate-Vocal-Remover/main/v'
model_ver = 'https://raw.githubusercontent.com/NaJeongMo/Colaboratory-Notebook-for-Ultimate-Vocal-Remover/main/model_list'

class hide_opt: # hide outputs
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def get_size(bytes, suffix='B'): # read ram
    global svmem
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f'{bytes:.2f}{unit}{suffix}'
        bytes /= factor
    svmem = psutil.virtual_memory()

def validateModelLinks():
    with hide_opt():
        get_ipython().system('wget {} -O model_ver'.format(model_ver))
    model_ver_ = open("model_ver", "r")
    model_ver_ = model_ver_.read()
    with hide_opt():
        get_ipython().system('wget {} -O model_list'.format(model_ver_))
    model_list = open("model_list", "r")
    model_list = model_list.readlines()
    models = []
    for i in model_list:
        models.append(i)
    os.remove('model_ver')
    os.remove('model_list')
    return models

def installAI():
    print('Installing ai...', end=' ')
    get_ipython().system('git clone {} VocalRemover5-COLAB_arch'.format(ai))
    os.chdir('VocalRemover5-COLAB_arch')
    print('done')

    print('Downloading models...', end=' ')
    for i in validateModelLinks():
        with hide_opt():
            zname = sanitize_filename(os.path.basename(i))
            get_ipython().system("wget {}".format(i))
            get_ipython().system('unzip -o {}'.format(zname))
            os.remove(zname)
    print('done')

def dlvr(): # download vr to colab only
    print('Warning: changes will not be saved.')
    print('Downloading VR5...')

    os.chdir('/content')
    installAI()
    #os.chdir('/content/VocalRemover5-COLAB_arch')
    print ("Success!")

def check_update():
    if os.path.isdir(f'{mounting_path}/VocalRemover5-COLAB_arch'): # check update if ai installed is True
        os.chdir(f'{mounting_path}/VocalRemover5-COLAB_arch')
        print("Checking for updates...", end=" ")
        with hide_opt():
            get_ipython().system('wget {} -O check_ver'.format(vercheck))
        f = open("check_ver", "r")
        nver = f.read()
        f = open("v", "r")
        cver = f.read()
        if cver != nver or ForceUpdate:
            print('New update found! {}'.format(nver))
            choice = str(input('Do you want to update? y/n')).lower()
            if choice == 'y':
                os.chdir('../')
                print('Updating ai...',end=' ')
                with hide_opt():
                    get_ipython().system('git clone {} temp_VocalRemover5-COLAB_arch'.format(ai))
                get_ipython().system('cp -a temp_VocalRemover5-COLAB_arch/* VocalRemover5-COLAB_arch/')
                get_ipython().system('rm -rf temp_VocalRemover5-COLAB_arch')
                print('done')

                print('Downloading models...', end=' ')
                os.chdir('VocalRemover5-COLAB_arch')
                for i in validateModelLinks():
                    with hide_opt():
                        zname = sanitize_filename(os.path.basename(i))
                        get_ipython().system("wget {}".format(i))
                        get_ipython().system('unzip -o {}'.format(zname))
                        os.remove(zname)
                print('done')
                output.clear()
                os.remove(f'{mounting_path}/VocalRemover5-COLAB_arch/v')
                os.rename(f'{mounting_path}/VocalRemover5-COLAB_arch/check_ver',f'{mounting_path}/VocalRemover5-COLAB_arch/v')
                os.chdir(f'{mounting_path}/VocalRemover5-COLAB_arch') # just to make sure

            else:
                print('Skipping update.')
                os.remove(f'{mounting_path}/VocalRemover5-COLAB_arch/v')
                os.rename(f'{mounting_path}/VocalRemover5-COLAB_arch/check_ver',f'{mounting_path}/VocalRemover5-COLAB_arch/v')
                os.chdir(f'{mounting_path}/VocalRemover5-COLAB_arch') # just to make sure
        else:
            os.remove('check_ver')
            print('No update found.')
    else:
            if os.path.isdir('/content/VocalRemover5-COLAB_arch'):
                print("Success!")
            else:
                dlvr()

#-------------Script begin-------------
if os.path.isdir('/content/VocalRemover5-COLAB_arch') == False:
    if os.path.isdir('/content/VocalRemover5-COLAB_arch'):
        print('Success!')
    else:
        dlvr()


In [3]:

#----------DEFINE INFERENCE REQ------------
def crc32(fileName):
    with open(fileName, 'rb') as fh:
        hash = 0
        while True:
            s = fh.read(65536)
            if not s:
                break
            hash = zlib.crc32(s, hash)
        return "%08X" % (hash & 0xFFFFFFFF)
def YouTube(link, dl=True):
    inputsha = hashlib.sha1(bytes(link, encoding='utf8')).hexdigest() + '.wav'
    fmt = '251/140/250/139'
    opt = {'format': fmt, 'outtmpl': inputsha, 'updatetime': False, 'nocheckcertificate': True}
    if dl == True:
        print('YouTube link detected')
        print('Downloading...', end=' ')
    with hide_opt():
        with youtube_dl.YoutubeDL(opt) as ydl:
            global desc
            if dl == True:
                desc = ydl.extract_info(link, download=not os.path.isfile(inputsha))
            else:
                desc = ydl.extract_info(link, download=not True)

    titlename = sanitize_filename(desc['title'])
    if dl == True:
        print('done')
        print(titlename)
    if dl == True:
        return titlename, inputsha
    else:
        return titlename
def zipdir(folder, zipname): # LINUX CALL MODIFIED!!!
    if '.zip' in zipname:
        zipname = zipname.strip('.zip')
    get_ipython().system('zip -r {}.zip {}'.format(zipname,folder))
# dlFile(input,pretrained_model,isYouTube='http://' in input,export_as_mp3=export_as_mp3)
def dlFile(track,pretrained_model,isYouTube=False,export_as_mp3=False):
    modelname = os.path.splitext(os.path.basename(pretrained_model))[0]
    stems = [f'_{modelname}_Instruments',f'_{modelname}_Vocals']
    filename = os.path.splitext(os.path.basename(track))[0]
    if isYouTube:
        filename = YouTube(track,dl=False)
    if os.path.isdir(filename):
        shutil.rmtree(filename)
        os.mkdir(filename)
    else:
        os.mkdir(filename)
    if os.path.isfile(f'{filename}.zip'):
        os.remove(f'{filename}.zip')
    if export_as_mp3:
        os.chdir('separated/')
        for i in stems:
            wav_path = filename + i + '.wav'
            mp3_path = filename + i + '.mp3'
            get_ipython().system(f'ffmpeg -y -i "{wav_path}" -vn -ar 44100 -ac 2 -b:a 320k "{mp3_path}" -loglevel quiet')
        os.chdir('../')
        for move in stems:
            shutil.move('separated/' + filename + move + '.mp3',filename)
    else:
        for move in stems:
            shutil.move('separated/' + filename + move + '.wav',filename)
    with hide_opt():
        get_ipython().system(f'zip -r "{filename}.zip" "{filename}"')
    shutil.rmtree(filename)
    files.download(f'{filename}.zip')

In [4]:
# ------------VSC REWRITE
#type: ignore
if os.path.isfile('main.py') == False:
    if MountDrive:
        os.chdir(mounting_path + '/VocalRemover5-COLAB_arch')
    else:
        os.chdir('/content/VocalRemover5-COLAB_arch')

#parameter markdowns-----------------


ScanSeparatedFolder = False #@param {type:"boolean"}
#@markdown Convert all files in your tracks folder
convertAll = False #@param {type:"boolean"}
if convertAll:
    convertAll = '--convert_all'
else:
    convertAll = ''
#@markdown Model name (Upload your models in models folder)
pretrained_model = "HP2-4BAND-3090_4band_arch-500m_1.pth" #@param ["HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","HP-4BAND-V2_arch-124m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
#@markdown ---
#@markdown ### Arguments
window_size =  512#@param {type:"integer"}
if window_size < 320:
    print('Warning: window_size lower than 320.')
if window_size < 272:
    window_size = 272
#1band_sr32000_hl512.json  2band_44100_lofi.json  3band_44100_mid.json
#1band_sr44100_hl512.json  2band_48000.json	 4band_44100.json
#2band_32000.json	  3band_44100.json	 ensemble.json
parameter = "Auto detect" #@param ["Auto detect","1band_sr32000_hl512.json","1band_sr44100_hl512.json", "2band_32000.json" , "2band_44100_lofi.json", "2band_48000.json", "3band_44100.json", "3band_44100_mid.json", "3band_44100_msb2.json", "4band_44100.json", "4band_v2.json"]
parameter = "modelparams/" + parameter
high_end_process = 'mirroring' #@param ["none","mirroring", "mirroring2" , "bypass"]
aggressiveness = '0.3' #@param {type:"string"}
aggressiveness = float(aggressiveness)
#@markdown Mute low volume vocals
postprocess = False #@param {type: "boolean"}
if postprocess:
    threshold = 0.2 #@param {type:"number"}
    min_range = 64 #?param {type:"integer"}
    fade_size = 32 #?param {type:"integer"}
    if min_range < fade_size * 2:
        print('min_range must be greater than or equal fade_size * 2')
        print('Using default instead. (except threshold)')
        min_range = 32
        fade_size = 64
    postprocess = f"-p -thres {threshold} -mrange {min_range} -fsize {fade_size}"
else:
    postprocess = ""
#@markdown ### Architecture
nn_architecture = 'Auto detect' #@param ["Auto detect","default", "34 MB", "124 MB", "130 MB", "500 MB"]
if nn_architecture == '34 MB':
    nn_architecture = '33966KB'
elif nn_architecture == '124 MB':
    nn_architecture = '123821KB'
elif nn_architecture == '130 MB':
    nn_architecture = '129605KB'
elif nn_architecture == '500 MB':
    nn_architecture = '537238KB'
elif nn_architecture == 'default':
    pass
#@markdown ---
#@markdown ### Checkboxes
#@markdown Use GPU for faster conversion
gpu = True #@param {type: "boolean"}
if gpu == True:
    gpu = 0
else:
    gpu = -1
#@markdown Aggressively remove vocals from Instrumental
deepExtraction = False #@param {type:"boolean"}
if deepExtraction:
    deepExtraction = "-D"
else:
    deepExtraction = ""
#@markdown Flip Instruments and Vocals output (Only for Vocal Models)
isVocal = False #@param {type:"boolean"}
if isVocal:
    isVocal = '--isVocal'
else:
    isVocal = ''
#@markdown Hide warnings
suppress = True #@param {type:"boolean"}
if suppress:
    suppress = '--suppress'
else:
    suppress = ''
#@markdown Export spectogram image
output_image = False #@param {type: "boolean"}
if output_image:
    output_image = "-I"
else:
    output_image = ""
#@markdown perform Test Time Augmentation to improve the separation quality
tta = True #@param {type: "boolean"}
if tta:
    tta = "-t"
else:
    tta = ""
#@markdown Use custom arguments
useCustomArguments = False #@param {type: "boolean"}
CustomArguments = "-h" #@param {type:"string"}
#@markdown Download files
download = False #@param {type:"boolean"}
export_as_mp3 = False #@param {type:"boolean"}
#@markdown Use all model
model_version = "Don't use all model" #@param ["Don't use all model","v5_new", "v5", "all"]
# {none,v5,v5_new,all}
# automation
if nn_architecture == 'Auto detect':
    if 'arch-default' in pretrained_model:
        nn_architecture = 'default'
    elif 'arch-34m' in pretrained_model:
        nn_architecture = '33966KB'
    elif 'arch-124m' in pretrained_model:
        nn_architecture = '123821KB'
    elif 'arch-130m' in pretrained_model:
        nn_architecture = '129605KB'
    elif 'arch-500m' in pretrained_model:
        nn_architecture = '537238KB'
    else:
        print('Error! autoDetect_arch')
        print('Using 124 MB instead.')
        nn_architecture = '123821KB'
if parameter == 'modelparams/Auto detect':
    if '4band' in pretrained_model.lower():
        if 'v2' in pretrained_model.lower():
            parameter = 'modelparams/4band_v2.json'
        else:
            parameter = 'modelparams/4band_44100.json'
    elif '3band' in pretrained_model.lower():
        if 'msb2' in pretrained_model.lower():
            parameter = 'modelparams/3band_44100_msb2.json'
        else:
            parameter = 'modelparams/3band_44100.json'
    elif 'midside' in pretrained_model.lower():
        parameter = 'modelparams/3band_44100_mid.json'
    elif '2band' in pretrained_model.lower():
        if 'lofi' in pretrained_model.lower():
            parameter = 'modelparams/2band_44100_lofi.json'
        else:
            parameter = 'modelparams/2band_48000.json'
    else:
        print('Parameter auto detect failed, using 1band instead.')
        parameter = 'modelparams/1band_sr44100_hl512.json'
if '34m' in pretrained_model or '124m' in pretrained_model or '130m' in pretrained_model or '500m' in pretrained_model:
    pretrained_model = "models/v5_new/" + pretrained_model
else:
    pretrained_model = "models/v5/" + pretrained_model
    if os.path.isfile(pretrained_model) == False:
        print('========================================================')
        print('                Error model not found.')
        print('Custom models should be uploaded in "models/v5/" folder!')
        print('========================================================')
if 'https://' not in input:
    if ScanSeparatedFolder:
        if input in ''.join(glob.glob('separated/*')):
            input = 'separated/' + input
        else:
            print('File not found in separated folder.')
            input = 'tracks/' + input
    else:
        input = 'tracks/' + input
elif 'https://' in input and ScanSeparatedFolder:
    print('Skipping "Separated" folder scan since a link is given.')

if model_version == "Don't use all model":
    model_version = 'none'

# --------------AI----------------
start_time = time.time()
#window_size,os.path.splitext(os.path.basename(pretrained_model)[0]),os.path.splitext(os.path.basename(parameter)[0],settings_tta = 'True' if tta else 'False'
#settings_tta = 'True' if tta else 'False',os.path.splitext(os.path.basename(parameter)[0],os.path.splitext(os.path.basename(pretrained_model)[0]),window_size
settings_tta = 'True' if tta else 'False'
settings_deepExtraction = 'True' if deepExtraction else 'False'

print('Window size: {}'.format(window_size))
print('Model: {}'.format(os.path.splitext(os.path.basename(pretrained_model))[0]))
print('Parameter: {}'.format(os.path.splitext(os.path.basename(parameter))[0]))
print('Aggressiveness: {}'.format(aggressiveness))
print('High end process: {}'.format(high_end_process))

print('TTA: {}'.format(settings_tta))
print('Deep Extraction: {}'.format(settings_deepExtraction))
print()
if useCustomArguments == False:
    get_ipython().system(f'python3.8 main.py -i "{input}" {convertAll} --useAllModel "{model_version}" --model_params "{parameter}" -P "{pretrained_model}" -w {window_size} -H "{high_end_process}" --aggressiveness {aggressiveness} -n "{nn_architecture}" -g {gpu} {deepExtraction} {isVocal} {suppress} {output_image} {postprocess} {tta}')
    if download and convertAll:
        sys.exit("No no, this is not an error but downloading with convertAll is not yet possible. Please DON'T report this to me (Hv) or the server")
    if download:
        dlFile(input,pretrained_model,isYouTube='https://' in input,export_as_mp3=export_as_mp3)
if useCustomArguments:
    get_ipython().system(f'python3.8 main.py {CustomArguments}')
print('Notebook took: {0:.{1}f}s'.format(time.time() - start_time, 1))


Window size: 512
Model: HP2-4BAND-3090_4band_arch-500m_1
Parameter: 4band_44100
Aggressiveness: 0.3
High end process: mirroring
TTA: True
Deep Extraction: False

YouTube Link detected
Downloading... WARNING: [youtube] YouTube said: ERROR - Precondition check failed.
done
비비 - 밤양갱 | [TEXTED] BIBI | 가사(Lyrics)
loading model... done
stft of wave source... done
100%|███████████████████████████████████████████| 53/53 [00:15<00:00,  3.42it/s]
inverse stft of Instruments... done
inverse stft of Vocals... done
Total time: 70.7s
Notebook took: 73.7s


In [5]:
import os
os.chdir('/content/VocalRemover5-COLAB_arch')

In [6]:
# file zip and move folder
import zipfile
import shutil

save_path = '/content/dataset/'

if not os.path.isdir(save_path):
    os.makedirs(save_path)

os.chdir('/content/VocalRemover5-COLAB_arch/separated')
precessed_file_path = os.getcwd()
zip_file = zipfile.ZipFile('zipfile.zip', 'w')

for i, file in enumerate(os.listdir(precessed_file_path)):
    if 'Vocals.wav' in file:
        shutil.move(os.path.join(precessed_file_path, file), os.path.join(precessed_file_path, str(i)+'_Vocals.wav'))
        zip_file.write(str(i)+'_Vocals.wav')
zip_file.close()
shutil.move(os.path.join(precessed_file_path, 'zipfile.zip'), os.path.join(save_path, 'zipfile.zip'))

'/content/dataset/zipfile.zip'

# TEST_Data Argumentation

# Voice Model Training

## Dependencies

In [28]:
import Config
config = Config()
experiment_name = config.experiment_name #@param {type:"string"}
print('experiment_name:', experiment_name)
dataset = config.dataset  #@param {type:"string"}
print('dataset:', dataset)

#@title Clone Repositories
import os

firsttry = True

# READ ME BEFORE CHANGING THINGS
# If you're attempting to replace the imports here with Applio-RVC, it will not work due to requirement discrepancies across the entire notebook.
# I will not be porting this notebook to Applio due to the failure of the Applio team to provide backwards compatibility with the Crepe and Mangio-Crepe f0 feature format.
# DO NOT ASK. IT WILL NOT HAPPEN.

os.chdir('/content/')

#if(os.path.exists("/content/Mangio-RVC-Fork")):
#  print("RVC already installed, skipping.")
#else:
#  #Credit to miaaaa0a on the AI Hub Discord for (indirectly) suggesting this variant of Mangio RVC to me.
#  !git clone -b pr-optimization --single-branch https://github.com/alexlnkp/Mangio-RVC-Tweaks.git
#  #Rename to keep backwards compatibility with old variants of Disconnected
#  os.rename("/content/Mangio-RVC-Tweaks", "/content/Mangio-RVC-Fork")
#  !git clone https://github.com/maxrmorrison/torchcrepe.git
#  !mv torchcrepe/torchcrepe Mangio-RVC-Fork/
#  !rm -rf torchcrepe  # Delete the torchcrepe repository folder

os.chdir('/content/Mangio-RVC-Fork')
now_dir = "/content/Mangio-RVC-Fork"
os.makedirs(os.path.join(now_dir, "logs"), exist_ok=True)
os.makedirs(os.path.join(now_dir, "weights"), exist_ok=True)

In [29]:
#@title GPU Check
import torch

ngpu = torch.cuda.device_count()
gpu_infos = []
mem = []
if_gpu_ok = False

if torch.cuda.is_available() or ngpu != 0:
  for i in range(ngpu):
    gpu_name = torch.cuda.get_device_name(i)
    if any(
        value in gpu_name.upper()
        for value in ["10", "16", "20", "30", "40", "A2", "A3", "A4", "P4", "A50", "500", "A60", "70", "80", "90", "M4", "T4", "TITAN"]
    ):
      if_gpu_ok = True
      print("Compatible GPU detected: %s" % gpu_name)
      gpu_infos.append("%s\t%s" % (i, gpu_name))
      mem.append(int(torch.cuda.get_device_properties(i).total_memory / 1024 / 1024 / 1024 + 0.4))

if if_gpu_ok and len(gpu_infos) > 0:
  gpu_info = "\n".join(gpu_infos)

else:
  raise Exception("No GPU detected; training cannot continue. Please change your runtime type to a GPU.")
gpus = "-".join(i[0] for i in gpu_infos)


Compatible GPU detected: NVIDIA GeForce RTX 3060


In [30]:
#@title Mount Drive
os.makedirs('/content/rvcDisconnected', exist_ok=True)

In [31]:
#@title Setup CSVDB
#...Alright, you made your point.
import csv

if not os.path.isdir("csvdb/"):
  os.makedirs("csvdb")
  frmnt, stp = open("csvdb/formanting.csv", "w", newline=""), open("csvdb/stop.csv", "w", newline="")
  csv_writer = csv.writer(frmnt, delimiter=",")
  csv_writer.writerow([False, 1.0, 1.0])
  csv_writer = csv.writer(stp, delimiter=",")
  csv_writer.writerow([False])
  frmnt.close()
  stp.close()

global DoFormant, Quefrency, Timbre
DoFormant, Quefrency, Timbre = False, 1.0, 1.0

In [36]:
#@title Set Training Variables
now_dir = "/content/Mangio-RVC-Fork"

pretrain_type = "OV2" #@param ["original", "OV2", "RIN_E3"] {allow-input: false}
path_to_training_folder = "/content/datasets/"
model_architecture = "v2" #@param ["v1","v2"] {allow-input: false}
target_sample_rate = "40k" #@param ["32k", "40k", "48k"] {allow-input: false}
#cpu_threads = 2 #@param {type:"integer"}
speaker_id = 0 #@param {type:"integer"}
pitch_extraction_algorithm = "rmvpe" #@param ["harvest", "crepe", "mangio-crepe", "rmvpe"] {allow-input: false}
crepe_hop_length = 64 #@param {type:"integer"}
pitch_guidance = True #@param {type:"boolean"}

cpu_threads = !nproc
cpu_threads = int(cpu_threads[0])

exp_dir = f"{now_dir}/logs/{experiment_name}"

assert crepe_hop_length!=None, "You need to input something for crepe_hop_length, silly."
assert crepe_hop_length>0, "Hop length must be more than 0."
assert crepe_hop_length<=512, "Save frequency must be less than 512."

if pretrain_type!="original" and model_architecture!="v2":
  model_architecture="v2"
  print("The new pretrains only support RVC v2 at this time. Your settings have been automatically adjusted.")

#TEMPORARY UNTIL SIMPLCUP 48K IS RELEASED
if pretrain_type!="original" and target_sample_rate=="48k":
  target_sample_rate="40k"
  print("The new pretrains only support 40k sample rate and lower at this time. Your settings have been automatically adjusted.")
if pretrain_type=="RIN_E3" and target_sample_rate!="40k":
  target_sample_rate="40k"
  print("RIN_E3 only supports 40k sample rate at this time. Your settings have been automatically adjusted.")


if(experiment_name == "experiment_name"):
  print("Warning: Your experiment name should be changed to the name of your dataset.")

## Preprocessing

In [37]:
#@title Load Dataset
#@markdown If it doesn't already exist, create a folder in your Google Drive named 'rvcDisconnected' and place your zip file there. This will look for the following ZIP file inside that 'rvcDisconnected' folder.


#@markdown This loader will load datasets in a similar fashion to the So-Vits-SVC dataset loader. For best results, your dataset should be formatted as such:
#@markdown ```
#@markdown zipfile.zip
#@markdown └───character_name
#@markdown     ├───file1.wav
#@markdown     ├───...
#@markdown     └───file999.wav
#@markdown ```
#@markdown Audio filenames do not matter. All audio files should be in WAV format for best compatibility.

# TODO: Add something to convert non-WAVs to WAV

import os
import shutil
import zipfile

directories=[]

def sanitize_directory(directory):
  for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path):
      if filename == ".DS_Store" or filename.startswith("._") or not filename.endswith(('.wav', '.flac', '.mp3', '.ogg', '.m4a')):
        os.remove(file_path)
    elif os.path.isdir(file_path):
      #Get rid of the MACOSX directory just so it doesn't mess with renaming later
      if(filename == "__MACOSX"):
        shutil.rmtree(file_path)
        continue
      #Append the directory to directories for future dataset check, then recurse.
      directories.append(file_path)
      sanitize_directory(file_path)

dataset_path = '/content/dataset/' + dataset
final_directory = '/content/datasets'
temp_directory = '/content/temp_dataset'

if os.path.exists(final_directory):
  print("Dataset folder already found. Wiping...")
  shutil.rmtree(final_directory)
if os.path.exists(temp_directory):
  print("Temporary folder already found. Wiping...")
  shutil.rmtree(temp_directory)

if not os.path.exists(dataset_path):
  raise Exception(f'I can\'t find {dataset} in {os.path.dirname(dataset_path)}.')

os.makedirs(final_directory, exist_ok=True)
os.makedirs(temp_directory, exist_ok=True)


#Oops.
os.system("unzip -d " + temp_directory + " -B " + dataset_path)
print("Sanitizing...")
sanitize_directory(temp_directory)

if(len(directories) == 0):
  #If there's no directories, we're dealing with a ZIP of just audio files.
  #Move everything to /dataset/experiment_name/.
  print("Dataset Type: Audio Files (Single Speaker)")
  expDir=os.path.join(final_directory, experiment_name)
  os.makedirs(expDir, exist_ok=True)
  for r, _, f in os.walk(temp_directory):
    for name in f:
      print("cp {}/{} {}".format(temp_directory, name, expDir))
      os.system("cp {}/{} {}".format(temp_directory, name, expDir))
elif(len(directories) == 1):
  #If there's only one directory, we're dealing with a single speaker.
  #Rename the folder to experiment_name and move it to /dataset/.
  print("Dataset Type: Single Speaker")
  fi = os.path.join(temp_directory, experiment_name)
  os.rename(directories[0], fi)
  shutil.move(fi, final_directory)

else:
  #If anything else, we're dealing with multispeaker.
  #Move all folders to /dataset/ indiscriminately.
  print("Dataset Type: Multispeaker")
  for fi in directories:
    shutil.move(fi, final_directory)

shutil.rmtree(temp_directory)

print("Dataset imported.")


Dataset folder already found. Wiping...
Archive:  /content/dataset/zipfile.zip
 extracting: /content/temp_dataset/2_Vocals.wav  
Sanitizing...
Dataset Type: Audio Files (Single Speaker)
cp /content/temp_dataset/2_Vocals.wav /content/datasets/experiment_name
Dataset imported.


## Preprocessing and Feature Extraction

In [38]:
#@title Preprocessing and Feature Extraction

import os
import subprocess

assert cpu_threads>0, "CPU threads not allocated correctly."

sr = int(target_sample_rate.rstrip('k'))*1000
pttf = path_to_training_folder + experiment_name
os.makedirs(f"{exp_dir}", exist_ok=True)

cmd = f"python trainset_preprocess_pipeline_print.py \"{pttf}\" {sr} {cpu_threads} \"{exp_dir}\" 1"
print(cmd)
os.system(cmd)

gpuList = gpus.split("-")
cmd = f"python extract_f0_print.py \"{exp_dir}\" {cpu_threads} {pitch_extraction_algorithm} {crepe_hop_length}"
print(cmd)
os.system(cmd)

leng = len(gpus)
cmd = f"python extract_feature_print.py \"device\" {leng} 0 0 \"{exp_dir}\" {model_architecture}"
print(cmd)
os.system(cmd)

python trainset_preprocess_pipeline_print.py "/content/datasets/experiment_name" 40000 12 "/content/Mangio-RVC-Fork/logs/experiment_name" 1
start preprocess
['trainset_preprocess_pipeline_print.py', '/content/datasets/experiment_name', '40000', '12', '/content/Mangio-RVC-Fork/logs/experiment_name', '1']


thread:0:   0%|          | 0/1 [00:00<?, ?it/s]
thread:1: 0it [00:00, ?it/s]


thread:2: 0it [00:00, ?it/s]



thread:3: 0it [00:00, ?it/s]




thread:4: 0it [00:00, ?it/s]





thread:5: 0it [00:00, ?it/s]






thread:6: 0it [00:00, ?it/s]







thread:7: 0it [00:00, ?it/s]








thread:8: 0it [00:00, ?it/s]









thread:9: 0it [00:00, ?it/s]










thread:10: 0it [00:00, ?it/s]











thread:11: 0it [00:00, ?it/s]
thread:0: 100%|██████████| 1/1 [00:34<00:00, 34.95s/it]


end preprocess
python extract_f0_print.py "/content/Mangio-RVC-Fork/logs/experiment_name" 12 rmvpe 64
['extract_f0_print.py', '/content/Mangio-RVC-Fork/logs/experiment_name', '12', 'rmvpe', '64']
Using f0 method: rmvpe


thread:0, f0ing, Hop-Length:64:   0%|          | 0/34 [00:00<?, ?it/s]


  0%|          | 0/34 [00:00<?, ?it/s]


thread:1, f0ing, Hop-Length:64:   0%|          | 0/34 [00:00<?, ?it/s]


  0%|          | 0/34 [00:00<?, ?it/s]


thread:3, f0ing, Hop-Length:64:   0%|          | 0/34 [00:00<?, ?it/s]



  0%|          | 0/34 [00:00<?, ?it/s]



thread:4, f0ing, Hop-Length:64:   0%|          | 0/34 [00:00<?, ?it/s]




  0%|          | 0/34 [00:00<?, ?it/s]




thread:5, f0ing, Hop-Length:64:   0%|          | 0/34 [00:00<?, ?it/s]





  0%|          | 0/33 [00:00<?, ?it/s]





thread:6, f0ing, Hop-Length:64:   0%|          | 0/33 [00:00<?, ?it/s]







  0%|          | 0/33 [00:00<?, ?it/s]







thread:8, f0ing, Hop-Length:64:   0%|          | 0/33 [00:00<?, ?it/s]



















  0%|          | 0/33 [00:00<?, ?it/s]










thread:11, f0ing, Hop-Length:64:   0%|          | 0/33 [00:00<?, ?it/s]









  0%|          | 0/33 [00:00<?, ?it/s]


















thread:9, f0ing, 

python extract_feature_print.py "device" 1 0 0 "/content/Mangio-RVC-Fork/logs/experiment_name" v2


2024-03-08 10:59:22 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


['extract_feature_print.py', 'device', '1', '0', '0', '/content/Mangio-RVC-Fork/logs/experiment_name', 'v2']
/content/Mangio-RVC-Fork/logs/experiment_name
load model(s) from hubert_base.pt


2024-03-08 10:59:22 | INFO | fairseq.tasks.hubert_pretraining | current directory is /content/Mangio-RVC-Fork
2024-03-08 10:59:22 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2024-03-08 10:59:22 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdrop': 0.05, 'dropout_input': 0.1, 'dropout_features': 0.1, 'fina

move model to cuda
all-feature-402
now-0,all-402,0_1.wav,(53, 768)
now-40,all-402,0_14.wav,(149, 768)
now-80,all-402,0_18.wav,(149, 768)
now-120,all-402,0_227.wav,(149, 768)
now-160,all-402,0_271.wav,(149, 768)
now-200,all-402,0_312.wav,(71, 768)
now-240,all-402,0_355.wav,(149, 768)
now-280,all-402,0_398.wav,(81, 768)
now-320,all-402,0_446.wav,(149, 768)
now-360,all-402,0_55.wav,(149, 768)
now-400,all-402,0_97.wav,(149, 768)
all-feature-done


0

# Training

### Load preprocessed dataset files from Google Drive

In [ ]:
##@title Load preprocessed dataset files from Google Drive (for resuming)
##@markdown If you already have preprocessed dataset files on Google Drive, you can load them here instead of re-running the preprocessing steps.
#import os
#
#BACK_UP_DATASET_PATH = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
#
##Prevent people from loading the ZIP over existing files
#ok=True
#if(os.path.exists("/content/Mangio-RVC-Fork/logs/"+experiment_name+"/2a_f0")):
#  print("Dataset files already loaded, skipping.")
#  ok=False
#
#if ok:
#  os.system("unzip {}/rvcLogs.zip -d /".format(BACK_UP_DATASET_PATH))

In [ ]:
##@title Import Model from Drive to Notebook (for resuming)
#import os
#
#DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
#DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
##@markdown Input the model's Step Count here (the number located on your model's G and D files.) If you used `save_only_last_ckpt` during training, this number will be 2333333.
#STEPCOUNT = 000 #@param {type:"integer"}
#
#print("Copying model files...")
#!cp "{DATASET_PATH_DRIVE}/D_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
#!cp "{DATASET_PATH_DRIVE}/G_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
#!cp "{DATASET_PATH_DRIVE}/config.json" "{DATASET_PATH_COLAB}"
#
#print("Copying Tensorboard TFEVENT files...")
#for r, _, f in os.walk(DATASET_PATH_DRIVE):
#  for name in f:
#    if(name.startswith("events.out.tfevents")):
#      !cp "{DATASET_PATH_DRIVE}/{name}" "{DATASET_PATH_COLAB}"
#
#print("All done. Welcome back!")

### Index Training

In [39]:
#@title Index Training
#@markdown Ensure that Feature Extraction has run successfully before running this cell.

#@markdown Use this option if you wish to save the two extra files generated by index training to your Google Drive. (Only the added index is normally needed.)
save_extra_files_to_drive = False #@param {type:"boolean"}

#Oh dear lord why is this baked into infer-web I hate this
import os
import sys
import traceback
import numpy as np
import faiss

#from sklearn.cluster import MiniBatchKMeans

exp_dir = "%s/logs/%s" % (now_dir, experiment_name)
os.makedirs(exp_dir, exist_ok=True)
feature_dir = (
    "%s/3_feature256" % (exp_dir)
    if model_architecture == "v1"
    else "%s/3_feature768" % (exp_dir)
)
print(feature_dir)
if not os.path.exists(feature_dir):
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")
listdir_res = list(os.listdir(feature_dir))
if len(listdir_res) == 0:
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")

try:
  from sklearn.cluster import MiniBatchKMeans
except:
  print("Due to a bug with Colab, we will need to reinstall Numpy real quick. Give me a sec!")
  os.system('pip install -U numpy')
  print("Numpy reinstalled. Please restart the runtime, and then re-run the \"Set Training Variables\" cell to continue.")
  sys.exit()
else:
  print("Proper Numpy version detected.")

infos=[]
npys=[]
for name in sorted(listdir_res):
  phone = np.load("%s/%s" % (feature_dir, name))
  npys.append(phone)
big_npy = np.concatenate(npys, 0)
big_npy_idx = np.arange(big_npy.shape[0])
np.random.shuffle(big_npy_idx)
if big_npy.shape[0] > 2e5:
  print("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
  try:
    big_npy = (
        MiniBatchKMeans(
            n_clusters=10000,
            verbose=True,
            batch_size=256,
            compute_labels = False,
            init="random"
        )
        .fit(big_npy)
        .cluster_centers_

    )
  except:
    info = traceback.format_exc()
    print(info)

np.save("%s/total_fea.npy" % exp_dir, big_npy)
n_ivf = min(int(16*np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
print("%s,%s" % (big_npy.shape, n_ivf))
index = faiss.index_factory(256 if model_architecture == "v1" else 768, "IVF%s,Flat" % n_ivf)
print("Training index...")
index_ivf = faiss.extract_index_ivf(index)
index_ivf.nprobe = 1
index.train(big_npy)
faiss.write_index(
    index,
    "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index" % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)
print("Adding...")
batch_size_add = 8192
for i in range(0, big_npy.shape[0], batch_size_add):
  index.add(big_npy[i:i+batch_size_add])
faiss.write_index(
    index,
    "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
    % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)

npr = index_ivf.nprobe

print("Saving files to Drive...")
DATASET_PATH_DRIVE = "/content/rvcDisconnected/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
    os.makedirs(DATASET_PATH_DRIVE)
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(save_extra_files_to_drive):
  os.system("cp {}/total_fea.npy {}".format(DATASET_PATH_COLAB, DATASET_PATH_DRIVE))
  os.system("cp {}/trained_IVF{}_Flat_nprobe_{}_{}_{}.index {}".format(DATASET_PATH_COLAB, n_ivf, npr, experiment_name, model_architecture, DATASET_PATH_DRIVE))
os.system("cp {}/added_IVF{}_Flat_nprobe_{}_{}_{}.index {}".format(DATASET_PATH_COLAB, n_ivf, npr, experiment_name, model_architecture, DATASET_PATH_DRIVE))

print("All done! Your index file has completed training.")
try:
  firsttry
except:
  print("If you had to restart the runtime, disconnect and delete the runtime in order to continue. (Restarting the runtime again will not work.)")



/content/Mangio-RVC-Fork/logs/experiment_name/3_feature768
Proper Numpy version detected.
(55456, 768),1421
Training index...
Adding...
Saving files to Drive...
All done! Your index file has completed training.


### Training

In [41]:
import os
import math
from random import shuffle

#@title Training
save_frequency = 10 #@param {type:"integer"}
total_epochs = 500 #@param {type:"integer"}
batch_size = 8 #@param {type:"integer"}
save_only_latest_ckpt = True #@param {type:"boolean"}
cache_all_training_sets = False #@param {type:"boolean"}
save_small_final_model = True #@param {type:"boolean"}
#@markdown The automatically calculated log interval is known to be very inaccurate and can cause delays between an epoch finishing and Tensorboard writes. If you would like, you can manually define a log interval here.
use_manual_stepToEpoch = False #@param {type:"boolean"}
manual_stepToEpoch = 000 #@param {type:"integer"}

assert save_frequency!=None, "You need to input something for save_frequency, silly."
assert save_frequency>0, "Save frequency must be more than 0."
if(save_frequency>50):print(f"...A save frequency of {save_frequency}? A bit high, but... alright then.")
assert total_epochs!=None, "You need to input something for total_epochs, silly."
assert total_epochs>0, "Total epochs must be more than 0."
if(total_epochs>10000):print(f"...A total epoch count of of {total_epochs}? This is going to overtrain, but... alright then.")
assert batch_size!=None, "You need to input something for batch_size, silly."
assert batch_size>0, "Batch size must be more than 0."
assert batch_size<=40, "Batch size must be less than 40. (I'd reccomend a value between 6 and 12 for Colab.)"

pretrained_base = "pretrained/" if model_architecture == "v1" else "pretrained_v2/"
unpt = f"_{pretrain_type}" if pretrain_type!="original" else ""

pretrainedD = f"{pretrained_base}f0D{target_sample_rate}{unpt}.pth"
pretrainedG = f"{pretrained_base}f0G{target_sample_rate}{unpt}.pth"

#Log interval
log_interval = 1
liFolderPath = os.path.join(exp_dir, "1_16k_wavs")
if(os.path.exists(liFolderPath) and os.path.isdir(liFolderPath)):
  wav_files = [f for f in os.listdir(liFolderPath) if f.endswith(".wav")]
  if wav_files:
    sample_size = len(wav_files)
    log_interval = math.ceil(sample_size / batch_size)
    if log_interval > 1:
      log_interval += 1

if log_interval > 250 and not use_manual_stepToEpoch:
  print(f"That's a big dataset you got there. Log interval normalized to 200 steps from {log_interval} steps.")
  log_interval = 200

if use_manual_stepToEpoch:
  log_interval = manual_stepToEpoch

#Create Python command
cmd = "python train_nsf_sim_cache_sid_load_pretrain.py -e \"%s\" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s -li %s" % (
    experiment_name,
    target_sample_rate,
    1,
    batch_size,
    0,
    total_epochs,
    save_frequency,
    "-pg %s" % pretrainedG if pretrainedG != "" else "\b",
    "-pd %s" % pretrainedD if pretrainedD != "" else "\b",
    1 if save_only_latest_ckpt else 0,
    1 if cache_all_training_sets else 0,
    1 if save_small_final_model else 0,
    model_architecture,
    log_interval,
)
print(cmd)

#Create mute filelist
gt_wavs_dir = f"{exp_dir}/0_gt_wavs"
feature_dir = (
  f"{exp_dir}/3_feature256"
  if model_architecture == "v1"
  else f"{exp_dir}/3_feature768"
)
f0_dir = f"{exp_dir}/2a_f0"
f0nsf_dir = f"{exp_dir}/2b-f0nsf"
names = (
  set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
  & set([name.split(".")[0] for name in os.listdir(feature_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
)
opt = []
for name in names:
  opt.append(
    "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
    % (
      gt_wavs_dir.replace("\\", "\\\\"),
      name,
      feature_dir.replace("\\", "\\\\"),
      name,
      f0_dir.replace("\\", "\\\\"),
      name,
      f0nsf_dir.replace("\\", "\\\\"),
      name,
      speaker_id,
    )
  )
fea_dim = 256 if model_architecture == "v1" else 768
for _ in range(2):
  opt.append(
      f"{now_dir}/logs/mute/0_gt_wavs/mute{target_sample_rate}.wav|{now_dir}/logs/mute/3_feature{fea_dim}/mute.npy|{now_dir}/logs/mute/2a_f0/mute.wav.npy|{now_dir}/logs/mute/2b-f0nsf/mute.wav.npy|{speaker_id}"
  )
shuffle(opt)
with open(f"{exp_dir}/filelist.txt", "w") as f:
  f.write("\n".join(opt))
print("Mute filelist written. Best of luck training!")


os.system("cd /content/Mangio-RVC-Fork")
os.system("load_ext tensorboard")
os.system("tensorboard --logdir /content/Mangio-RVC-Fork/logs")

os.chdir('/content/Mangio-RVC-Fork')
os.system(cmd)


python train_nsf_sim_cache_sid_load_pretrain.py -e "experiment_name" -sr 40k -f0 1 -bs 8 -g 0 -te 500 -se 10 -pg pretrained_v2/f0G40k_OV2.pth -pd pretrained_v2/f0D40k_OV2.pth -l 1 -c 0 -sw 1 -v v2 -li 52
Mute filelist written. Best of luck training!


sh: 1: load_ext: not found
TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.14.0 at http://localhost:6006/ (Press CTRL+C to quit)


### Export Model from Notebook to Drive

In [ ]:
#@title Export Model from Notebook to Drive
import os

DATASET_PATH_DRIVE = "/content/rvcDisconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
  os.makedirs(DATASET_PATH_DRIVE)

#@markdown Use this option if you wish to only copy over weights.
skip_models = False #@param {type:"boolean"}
#@markdown Use these options if you wish to manually input your step count and epoch count for incomplete models. *Do not use this option if your training finished.*
manual_save = False #@param {type:"boolean"}
STEPCOUNT = 000 #@param {type:"integer"}
EPOCHCOUNT = 000 #@param {type:"integer"}

finished=False
potential="/content/Mangio-RVC-Fork/weights/"+experiment_name+".pth"
if os.path.exists(potential):
  finished = True

#VERY hacky. Might break stuff, report to me if it does.
print("Detecting latest model...")
if(not manual_save):
  currentMax = 0
  for r, _, f in os.walk("/content/Mangio-RVC-Fork/weights/"):
    for name in f:
      if(name.endswith(".pth") and (name!=experiment_name+".pth")):
        #Check to see if this PTH is what we're looking for.
        if(name.find(experiment_name)==-1):
          continue
        #Determine Epochcount+Stepcount Phase 1
        pot = name.split('_')
        ep=pot[len(pot)-2][1:]
        #If what we got from the epoch count section of the filename isn't a number, multiple completed models are in weights.
        #Skip it if that happens.
        if(not ep.isdecimal()):
          continue
        #Determine Epochcount+Stepcount Phase 2
        ep=int(ep)
        if ep>currentMax:
          currentMax=ep
          step=pot[len(pot)-1].split('.')
          step=int(step[0][1:])
          EPOCHCOUNT=ep
          STEPCOUNT=step

TSTEP = STEPCOUNT
if(not skip_models):
  print("Copying model files...")
  if(save_only_latest_ckpt):
    TSTEP=2333333
  os.system("cp {}/D_{}.pth {}".format(DATASET_PATH_COLAB, TSTEP, DATASET_PATH_DRIVE))
  os.system("cp {}/G_{}.pth {}".format(DATASET_PATH_COLAB, TSTEP, DATASET_PATH_DRIVE))
  os.system("cp {}/config.json {}".format(DATASET_PATH_COLAB, DATASET_PATH_DRIVE))

print("Copying Tensorboard TFEVENT files...")
for r, d, f in os.walk(DATASET_PATH_COLAB):
  for name in f:
    if(name.startswith("events.out.tfevents") and os.path.exists(os.path.join(DATASET_PATH_COLAB, name))):
      os.system("cp {}/{} {}".format(DATASET_PATH_COLAB, name, DATASET_PATH_DRIVE))

print("Copying weight file...")
if(finished):
  os.system("cp {} {}".format(potential, DATASET_PATH_DRIVE))
else:
  os.system("cp /content/Mangio-RVC-Fork/weights/{}_e{}_s{}.pth {}".format(experiment_name, EPOCHCOUNT, STEPCOUNT, DATASET_PATH_DRIVE))

print("All done!")